In [16]:
%%configure -f
{
    "conf": {
        "spark.jars": "s3://aws-emr-resources-322477792531-us-east-1/graphframes-0.7.0-spark2.4-s_2.11.jar",
        "spark.pyspark.python": "python3",
        "spark.pyspark.virtualenv.enabled": "true",
        "spark.pyspark.virtualenv.type":"native",
        "spark.pyspark.virtualenv.bin.path":"/usr/bin/virtualenv"
    }
}

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
1,application_1654117362675_0002,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
1,application_1654117362675_0002,pyspark,idle,Link,Link,None,✔


In [17]:
sc.install_pypi_package('graphframes')
sc.install_pypi_package('boto3')
sc.install_pypi_package('pandas==1.0.3')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  Using cached https://files.pythonhosted.org/packages/0b/27/c7c7e1ced2fe9a905f865dd91faaec2ac8a8e313f511678c8ec92a41a153/graphframes-0.6-py2.py3-none-any.whl

  Using cached https://files.pythonhosted.org/packages/ce/3d/dc2fedf59cac4c7bd60362ccea1c1b299bb7952a9c2d563a7277a0c69e93/boto3-1.24.1-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/5e/c6/af903b5fab3f9b5b1e883f49a770066314c6dcceb589cf938d48c89556c1/s3transfer-0.6.0-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/76/3a/cc4f627b725d50a9f8c202eb15a2256ac5c324588e8975eb42937b9ba78d/botocore-1.27.1-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/ec/03/062e6444ce4baf1eac17a6a0ebfe36bb1ad05e1df0e20b110de59c278498/urllib3-1.26.9-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/36/7a/87837f39d0296e723bb9b62bbb257d0355c7f6128853c78955f57342a56d/python_dateutil-2.8.2-py2.py3-none-any.whl

  Using cached https://files.pythonhosted.org/package

In [18]:
import boto3
import pandas as pd
s3 = boto3.client('s3')
obj = s3.get_object(Bucket='crypto-conven-training', Key='var_network/network_short_freq.csv')
short_freq = pd.read_csv(obj['Body'], index_col=0)
obj = s3.get_object(Bucket='crypto-conven-training', Key='var_network/network_medium_freq.csv')
medium_freq = pd.read_csv(obj['Body'], index_col=0)
obj = s3.get_object(Bucket='crypto-conven-training', Key='var_network/network_long_freq.csv')
long_freq = pd.read_csv(obj['Body'], index_col=0)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
# use short-frequency network as an example to conduct following analysis
nodes = pd.DataFrame({'id': short_freq.index, 'type': ['crypto' if i < 103 else 'conven' for i in range(len(short_freq.index))]})
edges = pd.DataFrame(columns=['src', 'dst', 'spillover', 'type'])
for index, row in short_freq.iterrows():
    for curr in row.index:
        if nodes[nodes.id == curr].type.values[0] == nodes[nodes.id == index].type.values[0]:
            rel = 'within'
        else:
            rel = 'cross'
        edges = edges.append({'src': curr, 'dst': index, 'spillover': short_freq.loc[index, curr], 'type': rel},
                              ignore_index=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
nodes = spark.createDataFrame(nodes)
edges = spark.createDataFrame(edges)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
from graphframes import *
g = GraphFrame(nodes, edges)
print(g)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

GraphFrame(v:[id: string, type: string], e:[src: string, dst: string ... 2 more fields])

In [22]:
sc.install_pypi_package('networkx')
sc.install_pypi_package("matplotlib==3.2.1")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  Using cached https://files.pythonhosted.org/packages/e9/93/aa6613aa70d6eb4868e667068b5a11feca9645498fd31b954b6c4bb82fa5/networkx-2.6.3-py3-none-any.whl

  Using cached https://files.pythonhosted.org/packages/b2/c2/71fcf957710f3ba1f09088b35776a799ba7dd95f7c2b195ec800933b276b/matplotlib-3.2.1-cp37-cp37m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/6c/10/a7d0fa5baea8fe7b50f448ab742f26f52b80bfca85ac2be9d35cdd9a3246/pyparsing-3.0.9-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/5c/f9/695d6bedebd747e5eb0fe8fad57b72fdf25411273a39791cde838d5a8f51/cycler-0.11.0-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/51/50/9a9a94afa26c50fc5d9127272737806990aa698c7a1c220b8e5075e70304/kiwisolver-1.4.2-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/75/e1/932e06004039dd670c9d5e1df0cd606bf46e29a28e65d5bb28e894ea29c9/typing_extensions-4.2.0-py3-none-any.whl

You are 

In [23]:
# import networkx as nx
# import matplotlib.pyplot as plt

# # G=nx.DiGraph()

# # for row in g.edges.select('src', 'dst', 'spillover').collect():
# #     if row['src'] != row['dst']:
# #         G.add_edge(row['src'], row['dst'], weight=row['spillover'])

# plt.figure(figsize=(10, 10))
# pos = nx.spring_layout(G)
# d = dict(G.degree(weight='weight'))
# nx.draw_networkx_labels(G, pos=pos, font_weight='bold')
# nx.draw_networkx_nodes(G, pos=pos, node_color='skyblue')
# nx.draw_networkx_edges(G, pos=pos, alpha=0.5, width=5)
# plt.close()

# %matplot plt

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
# Query: Get in-degree of each vertex.
g.inDegrees.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+--------+
|      id|inDegree|
+--------+--------+
| USDPINK|     155|
| USDOMNI|     155|
|  USDUBQ|     155|
| USDDASH|     155|
|  USDDZD|     155|
|  USDPYG|     155|
|  USDPOT|     155|
| USDBELA|     155|
|  USDMEC|     155|
|  USDNYC|     155|
|  USDNOK|     155|
|  USDXPF|     155|
|  USDSYS|     155|
|  USDEMC|     155|
|  USDEUR|     155|
|  USDETH|     155|
| USDBITB|     155|
|  USDMDL|     155|
|USDCLOAK|     155|
|  USDXST|     155|
+--------+--------+
only showing top 20 rows

In [25]:
# Query: Count the number of "follow" connections in the graph.
g.edges.filter("type = 'within' and spillover > 0.2").count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2977

In [26]:
# Query: Count the number of "follow" connections in the graph.
g.edges.filter("type = 'cross' and spillover > 0.2").count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

883

In [27]:
# Filter
g_cryp_within = g.filterEdges("type = 'within' and spillover > 0.2").filterVertices("type = 'crypto'").dropIsolatedVertices()
g_conv_within = g.filterEdges("type = 'within' and spillover > 0.2").filterVertices("type = 'conven'").dropIsolatedVertices()
g_cross = g.filterEdges("type = 'cross' and spillover > 0.2").dropIsolatedVertices()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [28]:
# pagerank calculation
results = g_cryp_within.pageRank(resetProbability=0.15, tol=0.01)
results.vertices.select("id", "pagerank").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+-------------------+
|      id|           pagerank|
+--------+-------------------+
| USDPINK|  1.078294976893571|
| USDOMNI| 0.8282832329425606|
|  USDUBQ| 1.5658873674698244|
| USDDASH| 0.8878214996299993|
|  USDPOT| 0.7251586293239374|
| USDBELA| 1.2786935329123064|
|  USDMEC| 0.5085845188878885|
|  USDNYC| 0.6044383593422594|
|  USDSYS| 1.1603157760657121|
|  USDEMC|  1.282685092679483|
|USDCLOAK| 0.9403953723306263|
|  USDXST|  0.627243422365171|
| USDTIPS| 0.7865072224070426|
|  USDNMC| 0.5435060596448064|
|  USDUNO| 1.5005494172403766|
|  USDTRC| 1.0643004312565711|
|  USDBLK|  1.177458950610755|
|  USDFTC|0.28610835924235484|
|  USDPIN| 0.4555727955267072|
|  USDBTS| 0.8883568877408112|
+--------+-------------------+
only showing top 20 rows

In [29]:
# pagerank calculation
results = g_conv_within.pageRank(resetProbability=0.15, tol=0.01)
results.vertices.select("id", "pagerank").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+-------------------+
|    id|           pagerank|
+------+-------------------+
|USDDZD|  1.012484743554313|
|USDPYG| 0.7594988259771634|
|USDNOK| 0.5720141510489021|
|USDXPF| 0.6554253939680479|
|USDEUR| 1.1216472323602182|
|USDMDL| 0.7894862784121143|
|USDCOP| 0.5538670370293423|
|USDURY|0.43880992573410316|
|USDMGA| 1.1752169679089284|
|USDCLP| 1.1635106349820832|
|USDRON| 0.6512691947199639|
|USDNAD| 1.6008673770090465|
|USDNZD| 1.7796556596529998|
|USDMXN| 0.5670056123428641|
|USDBRL| 0.3540187858969926|
|USDRUB| 1.1916012427997498|
|USDSTD| 1.1772318355872733|
|USDMKD| 0.8014136816491488|
|USDCRC| 0.7740156789692019|
|USDFJD|   1.03182083232021|
+------+-------------------+
only showing top 20 rows

In [30]:
results.edges.select("src", "dst", "weight").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+------+--------------------+
|   src|   dst|              weight|
+------+------+--------------------+
|USDDZD|USDGBP| 0.05555555555555555|
|USDDZD|USDEUR| 0.05555555555555555|
|USDDZD|USDNZD| 0.05555555555555555|
|USDDZD|USDZAR| 0.05555555555555555|
|USDDZD|USDSZL| 0.05555555555555555|
|USDDZD|USDSTD| 0.05555555555555555|
|USDDZD|USDMAD| 0.05555555555555555|
|USDDZD|USDBWP| 0.05555555555555555|
|USDDZD|USDKMF| 0.05555555555555555|
|USDDZD|USDDZD| 0.05555555555555555|
|USDDZD|USDUGX| 0.05555555555555555|
|USDDZD|USDNOK| 0.05555555555555555|
|USDDZD|USDUAH| 0.05555555555555555|
|USDDZD|USDMDL| 0.05555555555555555|
|USDDZD|USDISK| 0.05555555555555555|
|USDDZD|USDCAD| 0.05555555555555555|
|USDDZD|USDPYG| 0.05555555555555555|
|USDDZD|USDCOP| 0.05555555555555555|
|USDPYG|USDGBP|0.047619047619047616|
|USDPYG|USDMGA|0.047619047619047616|
+------+------+--------------------+
only showing top 20 rows